Mock dataset just to see if multi-system integration works. E.g. using mouse/human subset, no proper gene matching, no pig HVGs,...

In [1]:
import scanpy as sc
import pandas as pd
import numpy as np

In [2]:
# Tutorial mouse&human data
adata_mmhs=sc.read('../../tutorials/data/mouse-human_pancreas_subset10000.h5ad')

In [3]:
adata_mmhs

AnnData object with n_obs × n_vars = 10000 × 1768
    obs: 'batch', 'mm_study', 'mm_sex', 'mm_age', 'mm_study_sample_design', 'mm_hc_gene_programs_parsed', 'mm_leiden_r1.5_parsed', 'cell_type_eval', 'system', 'hs_Sex', 'hs_Diabetes Status', 'leiden_system'
    var: 'gs_mm', 'gs_hs'
    obsm: 'X_pca_system'
    layers: 'counts'

In [27]:
# pig data full
adata_sd=sc.read('/om2/user/khrovati/data/datasets/d10_1016_j_molmet_2022_101595/15202490-e1d4-43ee-bb0d-f3494f21fba2.h5ad')

In [28]:
adata_sd

AnnData object with n_obs × n_vars = 22056 × 15824
    obs: 'mt_frac', 'n_counts', 'n_genes', 'sample', 'size_factors', 'doublet_score', 'louvain_anno_broad', 'assay_ontology_term_id', 'cell_type_ontology_term_id', 'development_stage_ontology_term_id', 'organism_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'is_primary_data', 'sex_ontology_term_id', 'tissue_ontology_term_id', 'donor_id', 'suspension_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage'
    var: 'pig_gene_names', 'feature_is_filtered', 'pig_ensembl_ID', 'feature_name', 'feature_reference', 'feature_biotype'
    uns: 'batch_condition', 'default_embedding', 'schema_version', 'title'
    obsm: 'X_pca', 'X_umap'

In [29]:
# prep pig data
adata_sd=adata_sd[np.random.RandomState(seed=0).permutation(adata_sd.obs_names)[:5000],:]
adata_sd=adata_sd.raw.to_adata()
adata_sd=adata_sd[:,np.array((adata_sd.X>0).sum(axis=0)>20).ravel()]
sc.pp.normalize_total(adata_sd, target_sum=1e4)
sc.pp.log1p(adata_sd)
adata_sd.obs['cell_type_eval']=adata_sd.obs['cell_type'].map({
    'type B pancreatic cell':'beta',
    'pancreatic A cell':'alpha', 
    'pancreatic D cell':'delta', 
    'pancreatic PP cell':'gamma'
})
adata_sd.obs['system']=2
adata_sd.obs['batch']=adata_sd.obs['sample']
adata_sd.obs=adata_sd.obs[['cell_type_eval','system','batch']]
adata_sd.var['gs_sd']=adata_sd.var['feature_name']
adata_sd.var=adata_sd.var[['gs_sd']]
adata_sd.var_names=adata_sd.var['gs_sd']
del adata_sd.uns
del adata_sd.obsm

/om2/user/khrovati/miniconda/envs/csi/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:170: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/om2/user/khrovati/miniconda/envs/csi/lib/python3.9/site-packages/anndata/_core/anndata.py:879: UserWarning: 
AnnData expects .var.index to contain strings, but got values like:
    ['PSMB1', 'FAM120B', 'DLL1', 'ERMARD', 'PHF10']

    Inferred to be: categorical

  names = self._prep_dim_index(names, "var")


In [26]:
# matching based on human gene names
adata_mmhs.var_names=adata_mmhs.var['gs_hs']

In [32]:
adata=sc.concat([adata_mmhs,adata_sd])

In [33]:
adata

AnnData object with n_obs × n_vars = 15000 × 1003
    obs: 'batch', 'cell_type_eval', 'system'

In [34]:
adata.write('/om2/user/khrovati/data/cross_system_integration/test_data/pancreas_3species_15000.h5ad')